## 环境配置

> 此为在线运行平台配置python3.9的指南，如在其他环境平台运行案例，请根据实际情况修改如下代码

第一步：设置python版本为3.9.0

In [ ]:
%%capture captured_output
!/home/ma-user/anaconda3/bin/conda create -n python-3.9.0 python=3.9.0 -y --override-channels --channel https://mirrors.tuna.tsinghua.edu.cn/anaconda/pkgs/main
!/home/ma-user/anaconda3/envs/python-3.9.0/bin/pip install ipykernel

In [ ]:
import json
import os

data = {
   "display_name": "python-3.9.0",
   "env": {
      "PATH": "/home/ma-user/anaconda3/envs/python-3.9.0/bin:/home/ma-user/anaconda3/envs/python-3.7.10/bin:/modelarts/authoring/notebook-conda/bin:/opt/conda/bin:/usr/local/nvidia/bin:/usr/local/cuda/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/home/ma-user/modelarts/ma-cli/bin:/home/ma-user/modelarts/ma-cli/bin"
   },
   "language": "python",
   "argv": [
      "/home/ma-user/anaconda3/envs/python-3.9.0/bin/python",
      "-m",
      "ipykernel",
      "-f",
      "{connection_file}"
   ]
}

if not os.path.exists("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/"):
    os.mkdir("/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/")

with open('/home/ma-user/anaconda3/share/jupyter/kernels/python-3.9.0/kernel.json', 'w') as f:
    json.dump(data, f, indent=4)

#### 注：以上代码运行完成后，需要重新设置kernel为python-3.9.0

<div align=center><img src="https://mindspore-demo.obs.cn-north-4.myhuaweicloud.com/imgs/ai-gallery/change-kernel.PNG"></div>

第二步：安装MindSpore框架和MindNLP套件

mindspore官网提供了不同的mindspore版本，可以根据自己的操作系统和Python版本，安装不同版本的mindspore


https://www.mindspore.cn/install

In [ ]:
%pip install https://ms-release.obs.cn-north-4.myhuaweicloud.com/2.5.0/MindSpore/unified/x86_64/mindspore-2.5.0-cp39-cp39-linux_x86_64.whl --trusted-host ms-release.obs.cn-north-4.myhuaweicloud.com -i https://pypi.tuna.tsinghua.edu.cn/simple

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.4/958.4 MB 9.1 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 16.8 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 14.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 16.6 MB/s eta 0:00:0000:0100:01
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
auto-tune 0.1.0 requires te, which is not installed.
schedule-search 0.0.1 requires absl-py, which is not installed.
Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install mindnlp==0.4.0 -i https://mirrors.aliyun.com/pypi/simple


Looking in indexes: https://mirrors.aliyun.com/pypi/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 4.0 MB/s eta 0:00:00a 0:00:010m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.2/19.2 MB 15.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for jieba: filename=jieba-0.42.1-py3-none-any.whl size=19314508 sha256=30064bba508d12a9c2c545bdec7e271f61d5a83e9fdd53298a82e74659e1fd26
  Stored in directory: /home/jiangna1/.cache/pip/wheels/95/ef/7c/d8b3108835edfa15487417c5bddff166482b195d8090117ac5
Successfully built jieba
Note: you may need to restart the kernel to use updated packages.


## Qwen微调推理全流程

# Qwen介绍

Qwen是由阿里巴巴云研发的大型语言模型。Qwen系列模型在多个基准测试中展现了卓越的性能，包括但不限于语言理解、数学计算、代码编写等方面。特别是在中文相关的任务上，其表现尤为突出，超过了其他一些知名的开源模型。Qwen在设计时充分考虑了中文用户的需求，具有强大的中文处理能力。它不仅能够理解和生成高质量的中文文本，还针对中文的语言特点进行了优化。在训练数据方面更侧重中文语料，针对中文nlp任务进行了优化，以及在推理方面表现出很强的能力。Qwen不仅仅局限于单一的应用领域，而是覆盖了从语言处理到视觉智能等多个方面，为开发者提供了广泛的应用场景。无论是AI绘画、AI写作还是编程辅助，Qwen都能提供有效的支持。


导入必要的包

In [1]:
import json
import numpy as np
import mindspore as ms
import mindspore.dataset as ds

/home/liangdeqi/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/liangdeqi/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/home/liangdeqi/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/home/liangdeqi/anaconda3/envs/py39/lib/python3.9/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [2]:
#将模式设置为动态图模式（PYNATIVE_MODE），并指定设备目标为Ascend芯片
ms.set_context(mode=ms.PYNATIVE_MODE, device_target="Ascend",device_id=5)


[WARNING] ME(3117682:281473615360032,MainProcess):2025-04-22-05:47:07.212.180 [mindspore/context.py:1335] For 'context.set_context', the parameter 'device_target' will be deprecated and removed in a future version. Please use the api mindspore.set_device() instead.
[WARNING] ME(3117682:281473615360032,MainProcess):2025-04-22-05:47:07.214.143 [mindspore/context.py:1335] For 'context.set_context', the parameter 'device_id' will be deprecated and removed in a future version. Please use the api mindspore.set_device() instead.


In [3]:
#指定模型路径
base_model_path = "/home/liangdeqi/liudengjin/Qwen2.5-3B/" #中文模型，这里我提前下载到了本地减少下载时间
# base_model_path = "NousResearch/Hermes-3-Llama-3.2-3B" #英文模型



## 数据集

这里提供两份用于微调的数据集，分别用于中文模型和英文模型，其中中文模型为hfl/chinese-llama-2-1.3b，参数量为1.3B，英文模型为NousResearch/Hermes-3-Llama-3.2-3B，参数量为3.2B，用户可以根据自己的配置或期望训练时长自行选择。

数据来源皆为hugging face公开用于微调的数据集，其中中文数据集来源为弱智吧，数据格式为：

  {"instruction": "只剩一个心脏了还能活吗？",

    "output": "能，人本来就只有一个心脏。"},

  {"instruction": "爸爸再婚，我是不是就有了个新娘？",

    "output": "不是的，你有了一个继母。\"新娘\"是指新婚的女方，而你爸爸再婚，他的新婚妻子对你来说是继母。"}


英文数据来源为Alpaca,数据格式为：

  {"instruction": "Give three tips for staying healthy.",

    "input": "",

    "output": "1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule."},

  {"instruction": "What are the three primary colors?",

    "input": "",

    "output": "The three primary colors are red, blue, and yellow."}


以下教程同时包括包括中文和英文模型的微调教程为例，其中英文模型微调效果更好，但因为时间关系，本模型展示主要以小规模的中文为例，可以自行根据自己的需求修改数据来源和模型。
 

### 数据加载和数据预处理

新建 tokenize_function 函数用于数据预处理，具体内容可见下面代码注释。

In [4]:
def tokenize_function(example, tokenizer):
    instruction = example.get("instruction", "")
    input_text = example.get("input", "")
    output = example.get("output", "")
    # prompt
    if input_text:
        prompt = f"User: {instruction} {input_text}\nAssistant: {output}"
    else:
        prompt = f"User: {instruction}\nAssistant: {output}"
    
    # Tokenize
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    input_ids = np.array(tokenized["input_ids"], dtype=np.int32)

    # Handle label
    pad_token_id = tokenizer.pad_token_id
    labels = np.array(
        [-100 if token_id == pad_token_id else token_id for token_id in input_ids], dtype=np.int32
    )
    return input_ids, labels



数据来源如下，为了避免网络问题，建议先下载到本地

https://huggingface.co/datasets/LooksJuicy/ruozhiba

In [5]:
# data_path = "/home/jiangna1/mindnlp_llama_all/alpaca_data.json" #英文数据集
data_path = "/home/liangdeqi/liudengjin/chinese_data.json"

查看数据具体内容，该数据只包括instruction和output两列

In [6]:
with open(data_path, 'r', encoding='utf-8') as f:
    data = json.load(f)
print(data[:3])

[{'instruction': '只剩一个心脏了还能活吗？', 'output': '能，人本来就只有一个心脏。'}, {'instruction': '爸爸再婚，我是不是就有了个新娘？', 'output': '不是的，你有了一个继母。"新娘"是指新婚的女方，而你爸爸再婚，他的新婚妻子对你来说是继母。'}, {'instruction': '樟脑丸是我吃过最难吃的硬糖有奇怪的味道怎么还有人买', 'output': '樟脑丸并不是硬糖，而是一种常见的驱虫药，不能食用。虽然它的味道可能不太好，但是由于其有效的驱虫效果，所以仍然有很多人会购买。'}]


从指定路径加载预训练的分词器，该分词器能将输入文本分割成模型可处理的词元。接着，将填充标记设置为结束标记，这样在处理不同长度的文本序列时，用结束标记来填充额外位置，避免引入额外特殊标记，减少模型学习负担。最后，设置填充方向为右侧，使文本在右侧添加填充标记达到统一长度，维持文本原始顺序。

In [7]:
from mindnlp.transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(base_model_path)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.pad_token_id = tokenizer.eos_token_id
tokenizer.padding_side = "right"
tokenizer.encode(' ;')


[WARNING] ME(3117682:281473615360032,MainProcess):2025-04-22-05:47:28.427.424 [mindspore/context.py:1335] For 'context.set_context', the parameter 'ascend_config' will be deprecated and removed in a future version. Please use the api mindspore.device_context.ascend.op_precision.precision_mode(),
                                                       mindspore.device_context.ascend.op_precision.op_precision_mode(),
                                                       mindspore.device_context.ascend.op_precision.matmul_allow_hf32(),
                                                       mindspore.device_context.ascend.op_precision.conv_allow_hf32(),
                                                       mindspore.device_context.ascend.op_tuning.op_compile() instead.
/home/liangdeqi/anaconda3/envs/py39/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  fr

[2587]

将数据分为训练集和验证集

In [8]:

def data_generator(dataset, tokenizer):
    for item in dataset:
        yield tokenize_function(item, tokenizer)
        
split_ratio = 0.9
split_index = int(len(data) * split_ratio)
train_data, val_data = data[:split_index], data[split_index:]

train_dataset = ds.GeneratorDataset(
    source=lambda: data_generator(train_data, tokenizer), 
    column_names=["input_ids", "labels"]
)

eval_dataset = ds.GeneratorDataset(
    source=lambda: data_generator(val_data, tokenizer), 
    column_names=["input_ids", "labels"]
)

查看处理后的数据，tokenizer 将输入的文本（prompt）拆分为词片段（tokens），然后将每个词片段映射为对应的 token ID。

In [10]:
for i, sample in enumerate(train_dataset.create_dict_iterator()):
    if i >= 5:
        break
    print(f"Sample {i}: Input IDs: {sample['input_ids'][:10]}") 
    print(f"Sample {i}: Labels: {sample['labels'][:10]}\n")   

[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB
Sample 0: Input IDs: [  1474     25  26853    103 100124  46944 103023  34187 104246  75606]
Sample 0: Labels: [  1474     25  26853    103 100124  46944 103023  34187 104246  75606]

Sample 1: Input IDs: [ 1474    25 10236   230   116 99962 87256 99838  3837 35946]
Sample 1: Labels: [ 1474    25 10236   230   116 99962 87256 99838  3837 35946]

Sample 2: Input IDs: [  1474     25   6567    101    253  99931 106256 104927 111505 116080]
Sample 2: Labels: [  1474     25   6567    101    253  99931 106256 104927 111505 116080]

Sample 3: Input IDs: [  1474     25  18137    102    105  17447  30534 107118 103009  99504]
Sample 3: Labels: [  1474     25  18137    102    105  17447  30534 107118 103009  99504]

Sample 4: Input IDs: [  1474     25    220 100678 106727  36587   1867   6484  24300   9370]
Sample 4: Labels: [  1474     25    220 100678 106727  36587   1867   6484  24300   9370]



## lora指令微调

指定微调结果输出路径

In [9]:
# 指定输出路径
peft_output_dir = "/home/liangdeqi/liudengjin/pert_model_Chinese"

加载基座模型


In [10]:
from mindnlp.transformers import AutoModelForCausalLM,  GenerationConfig

ms_base_model = AutoModelForCausalLM.from_pretrained(base_model_path, ms_dtype=ms.float16)
ms_base_model.generation_config = GenerationConfig.from_pretrained(base_model_path)
ms_base_model.generation_config.pad_token_id = ms_base_model.generation_config.eos_token_id


Qwen2ForCausalLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`.`PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Sliding Window Attention is enabled but not implemented for `eager`; unexpected results may be encountered.


[MS_ALLOC_CONF]Runtime config:  enable_vmm:True  vmm_align_size:2MB


Loading checkpoint shards: 100%|██████████| 2/2 [00:17<00:00,  8.75s/it]


In [11]:
#修改精度，会使训练变慢，但是训练loss下降效果会变好
for name, param in ms_base_model.parameters_and_names():
    param.set_dtype(ms.float32) 

这部分代码的主要作用是创建一个 LoRA的配置对象 ms_config，大语言模型进行微调时，LoRA 是一种高效的参数微调方法，通过在预训练模型的基础上添加低秩矩阵来减少需要训练的参数数量，从而提高微调效率。

In [12]:
from mindnlp.peft import LoraConfig, TaskType, get_peft_model, PeftModel

ms_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,#微调任务的类型
    #指定需要应用 LoRA 调整的目标模块
    # target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj"],
    inference_mode=False,  
    r=8,                   
    lora_alpha=32,         
    lora_dropout=0.1       
)


基于给定的基础模型和 LoRA 配置创建一个可进行参数高效微调的模型。

In [13]:
ms_model = get_peft_model(ms_base_model, ms_config)

## 微调
通过 BertForSequenceClassification 构建用于情感分类的 BERT 模型，加载预训练权重，设置情感三分类的超参数自动构建模型。后面对模型采用自动混合精度操作，提高训练的速度，然后实例化优化器，紧接着实例化评价指标，设置模型训练的权重保存策略，最后就是构建训练器，模型开始训练。

训练参数的设置

In [14]:
from mindnlp.engine import TrainingArguments, Trainer

num_train_epochs = 10
fp16 = True
overwrite_output_dir = True
per_device_train_batch_size = 2
per_device_eval_batch_size = 4
gradient_accumulation_steps = 16
gradient_checkpointing = True
evaluation_strategy = "steps"
learning_rate = 1e-5
lr_scheduler_type = "cosine" 
weight_decay = 0.01
warmup_ratio = 0.1
max_grad_norm = 0.3
group_by_length = False 
auto_find_batch_size = False
save_steps = 50 
logging_strategy = "steps"
logging_steps = 150                
load_best_model_at_end = True 
packing = False
save_total_limit = 3
neftune_noise_alpha = 5 
eval_steps = 10

training_arguments = TrainingArguments(
    output_dir=peft_output_dir,
    overwrite_output_dir=overwrite_output_dir,
    num_train_epochs=num_train_epochs,
    load_best_model_at_end=load_best_model_at_end,
    per_device_train_batch_size=per_device_train_batch_size,
    per_device_eval_batch_size=per_device_eval_batch_size,
    evaluation_strategy=evaluation_strategy,
    eval_steps=eval_steps,
    max_grad_norm=max_grad_norm,
    auto_find_batch_size=auto_find_batch_size,
    save_total_limit=save_total_limit,
    gradient_accumulation_steps=gradient_accumulation_steps,
    save_steps=save_steps,
    logging_strategy=logging_strategy,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type
)

初始化训练器

In [15]:
trainer = Trainer(
    model=ms_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=training_arguments
)

开始训练

In [18]:
trainer.train()




  0%|          | 0/420 [00:00<?, ?it/s][WARNING] PRE_ACT(2760802,ffff2d72f120,python):2025-04-22-01:39:51.736.461 [mindspore/ccsrc/backend/common/mem_reuse/mem_dynamic_allocator.cc:721] FreeIdleMemsByEagerFree] Eager free count : 2, free memory : 32172385280, real free : 6721372160, not free size: 25451013120.


.

  2%|▏         | 10/420 [02:06<1:20:24, 11.77s/it]We detected that you are passing `past_key_values` as a tuple and this is deprecated. Please use an appropriate `Cache` class

  2%|▏         | 10/420 [02:21<1:20:24, 11.77s/it]

{'eval_loss': 0.3680526614189148, 'eval_runtime': 14.2338, 'eval_samples_per_second': 2.67, 'eval_steps_per_second': 0.703, 'epoch': 0.24}


                                                  
  5%|▍         | 20/420 [04:31<1:19:11, 11.88s/it]

...

                                                   
 98%|█████████▊| 410/420 [1:33:26<02:02, 12.21s/it]

{'eval_loss': 0.36315032839775085, 'eval_runtime': 13.8025, 'eval_samples_per_second': 2.753, 'eval_steps_per_second': 0.725, 'epoch': 9.75}


                                                   
100%|██████████| 420/420 [1:35:37<00:00, 13.66s/it]

{'eval_loss': 0.36315810680389404, 'eval_runtime': 13.7993, 'eval_samples_per_second': 2.754, 'eval_steps_per_second': 0.725, 'epoch': 9.99}
{'train_runtime': 5737.5188, 'train_samples_per_second': 2.346, 'train_steps_per_second': 0.073, 'train_loss': 0.3477174577258882, 'epoch': 9.99}


TrainOutput(global_step=420, training_loss=0.3477174577258882, metrics={'train_runtime': 5737.5188, 'train_samples_per_second': 2.346, 'train_steps_per_second': 0.073, 'train_loss': 0.3477174577258882, 'epoch': 9.99})

In [16]:
for name, param in trainer.model.parameters_and_names():
    param.set_dtype(ms.float16)
trainer.model.save_pretrained(peft_output_dir)

最后存储还是存储为 16 位浮点数，保存训练后的 LoRA 模型保存到指定的输出目录

## 推理

注意，lora微调后保存的并不是完整的参数，在推理时，需要将保存的 LoRA 参数加载到原预训练模型中，合并后得到完整的模型，然后进行推理。

使用PeftModel进行配置和参数合并，最后将模型设置为评估模式，进行推理任务。

In [17]:
#将 LoRA微调后的参数加载到预训练模型中
from mindnlp.peft import PeftModel
model = PeftModel.from_pretrained(ms_base_model, peft_output_dir)
model = model.merge_and_unload()
print('model merge succeeded')
model.eval()


model merge succeeded


Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-35): 36 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): lora.Linear(
            (base_layer): Linear (2048 -> 2048)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear (2048 -> 8)
            )
            (lora_B): ModuleDict(
              (default): Linear (8 -> 2048)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear(
            (base_layer): Linear (2048 -> 256)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear (2048 -> 8)
            )
 

定义一个函数，用于根据用户输入的问题生成相应的回答

In [18]:
import mindspore as ms

def generate_response(question, model, tokenizer, max_length=256):
    prompt = f"以下是用户和助手之间的问答。\n问：{question}\n答："
    inputs = tokenizer(prompt, return_tensors="ms", padding=True, truncation=True, max_length=512)
    output_ids = model.generate(
        **inputs,
        do_sample=False,
        # temperature=0.7,
        # top_p=0.9,
        repetition_penalty=1.2,
        no_repeat_ngram_size=3,
        max_length=max_length,
        eos_token_id=tokenizer.eos_token_id
    )

    response = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    response = response.split("Answer：")[-1].strip()
    return response


一个实例

In [19]:
question = "如何保持清醒?"
response = generate_response(question, model, tokenizer)

print(f"User: {question}")
print(f"LLAMA: {response}")

Both `max_new_tokens` (=2048) and `max_length`(=256) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


User: 如何保持清醒?
LLAMA: 以下是用户和助手之间的问答。
问：如何保持清醒?
答：喝咖啡或茶，吃一些富含蛋白质的食物。
